# Can we make Python fast without sacrificing readability?

## numba for astrodynamics

![numba](img/orbit-low-thrust.png)

### Juan Luis Cano Rodríguez <hello@juanlu.space>
### 2019-09-05 EuroSciPy @ Bizkaia Aretoa

# Outline

1. Introduction
2. Why numba?
4. Tips, tricks, limitations and workarounds
5. Conclusions

# Who is this guy?

* **Aerospace Engineer** with a passion for orbits 🛰
* Chair of the **Python España** non profit and co-organizer of **PyCon Spain** 🐍
* **Software Developer** at **Satellogic** 🌍
* Free Software advocate and Python enthusiast 🕮
* Hard Rock lover 🎸

Follow me! https://github.com/Juanlu001/

![Me!](img/juanlu_esa.jpg)

# 1. What is poliastro?

<img src="img/logo_text.svg" alt="poliastro" width="500" style="float:right"></img>

> poliastro is Python library for Astrodynamics and Orbital Mechanics, focused on interactive and friendly use and with an eye on performance.

* **Pure Python**, accelerated with **numba**
* **MIT license** (permissive)
* Physical units, astronomical scales and more, thanks to Astropy
* Conversion between several orbit representations
* Analytical and numerical propagation
* Cool documentation 🚀
* Latest version 0.13.0 https://docs.poliastro.space/en/v0.13.0/changelog.html#poliastro-0-13-0-2019-08-05

<div style="clear:both"></div>

## Brief history

* **2013**: **First version: Octave + FORTRAN + Python**
* *2014*: Refactor of the API, much friendlier
* *2015*: Replace FORTRAN algorithms by Python + numba 🚀
* *2016*: Izzo algorithm (Lambert's problem), 6th ICATT @ ESA
* *2017*: Summer of Code in Space (SOCIS), OpenAstronomy & Astropy membership, 1st OSCW @ ESOC
* *2018*: Google Summer of Code (GSOC), #PyAstro18 @ Simons Fndn, expansion into the industry

# 2. Why numba? A shallow\* overview of Python accelerators

## Cython

![Cython](img/Cython-logo.png)

* Mature, widely used, effective, gradual - a great project!
* Some personal problems with it:
  - I don't know any C, so it's more difficult for me
  - I wanted poliastro to be super easy to install by avoiding the "two language" problem (this includes Windows)
  - The native debugger has been broken two years https://github.com/cython/cython/issues/1717

I don't have lots of experience with it, so I don't have solid arguments against it.

\* There are many more we left out

## PyPy

![PyPy](img/pypy-logo.png)

* PyPy is a super interesting alternative Python implementation https://pypy.org/
* I really really want to use it more, but there are some obstacles:
  - The documentation is a bit poor, even the changelogs
  - Lacks interest from the mainstream community (including snarky comments by Guido about "nobody using it in production")
  - Support in conda is not mature https://github.com/conda-forge/pypy2.7-feedstock/issues/1
  - PyPy has several incompatibilities with manylinux1 wheels https://bitbucket.org/pypy/pypy/issues/2617/
  - manylinux2010 are almost there, but need the final push https://github.com/pypa/manylinux/issues/179
  - Last time I tried, the test suite was actually slower 😢 https://github.com/poliastro/poliastro/issues/98#issuecomment-469597401

## All of a sudden, in 2012...

![numba 0.1 is released](img/tweet-travis.png)

https://twitter.com/teoliphant/status/235789560678858752

# numba

![numba](img/numba.png)

> Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.

* Latest version (at the time of writing) 0.45.1
* Documentation https://numba.pydata.org/numba-doc/0.45.1/index.html
* BSD-2 License
* Easy to install:

```
$ pip install numba
$ conda install numba [--channel conda-forge]
```

For more details, see [Valentin's talk](https://pretalx.com/euroscipy-2019/talk/EDNVGJ/) 😉

## And it worked!

"poliastro: An Astrodynamics library written in Python with Fortran performance" at the 6th International Conference on Astrodynamics Tools and Techniques

![poliastro benchmark](img/poliastro-benchmark.png)

https://indico.esa.int/event/111/contributions/393/

# 3. Tips, tricks, caveats and limitations

## The "nopython mode" is the only way

* Two modes: "object mode" and "nopython mode"
* Only the latter is truly optimized!
* Functions JITted in nopython mode can only call "nopython" functions
* _Avoid "object mode"!_ In the process of being deprecated

![It's nopython all the way down](img/nopython.jpg)

In [1]:
from numba import jit

In [2]:
@jit
def range10():
    l = []
    for x in range(10):
        l.append(x)
    return l

range10()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
@jit
def reversed_range10():
    l = []
    for x in range(10):
        l.append(x)

    return reversed(l)  # innocuous change, but no `reversed` support in nopython mode


list(reversed_range10())

<ipython-input-3-36e21b3924b1>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "reversed_range10" failed type inference due to: Untyped global name 'reversed': cannot determine Numba type of <class 'type'>

File "<ipython-input-3-36e21b3924b1>", line 7:
def reversed_range10():
    <source elided>

    return reversed(l)  # innocuous change, but no `reversed` support in nopython mode
    ^

  @jit
<ipython-input-3-36e21b3924b1>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "reversed_range10" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-3-36e21b3924b1>", line 4:
def reversed_range10():
    <source elided>
    l = []
    for x in range(10):
    ^

  @jit
/home/juanlu/.local/share/virtualenvs/talk-numba-astrodynamics-VOhN9a7V/lib/python3.7/site-packages/numba/compiler.py:742: NumbaWarning: Function "re

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [4]:
list(reversed_range10())

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [5]:
@jit(nopython=True)
def reversed_range10():
    l = []
    for x in range(10):
        l.append(x)

    return reversed(l)  # innocuous change, but no `reversed` support in nopython mode


list(reversed_range10())

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'reversed': cannot determine Numba type of <class 'type'>

File "<ipython-input-5-e3aec2acc4da>", line 7:
def reversed_range10():
    <source elided>

    return reversed(l)  # innocuous change, but no `reversed` support in nopython mode
    ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/latest/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#my-code-doesn-t-compile

If you think your code should work with Numba, please report the error message
and traceback, along with a minimal reproducer at:
https://gith

```
TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'reversed': cannot determine Numba type of <class 'type'>

File "<ipython-input-10-e3aec2acc4da>", line 7:
def reversed_range10():
    <source elided>

    return reversed(l)  # innocuous change, but no `reversed` support in nopython mode
    ^
```

In [6]:
from numba import njit

@njit
def reversed_range10():
    l = []
    for x in range(10):
        l.append(x)

    return l[::-1]  # alternative to `reversed`

reversed_range10()

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

## Passing functions as arguments is _slow_

* Since numba 0.38 the user can pass JITted functions as arguments, yay!
* However, the dispatch overhead for this case appears to be in the ~tens of microseconds range https://github.com/numba/numba/issues/2952
* Arguably the most important blocker to write reusable numba code

### Example: Root finding _à la_ SciPy

In [7]:
@njit
def func(x):
    return x**3 - 1

@njit
def fprime(x):
    return 3 * x**2

In [8]:
@njit
def njit_newton(func, x0, fprime):
    for _ in range(50):
        fder = fprime(x0)
        fval = func(x0)
        newton_step = fval / fder
        x = x0 - newton_step
        if abs(x - x0) < 1.48e-8:  # Harcoded value in scipy.optimize.newton
            return x
        x0 = x

In [9]:
%timeit njit_newton(func, 1.5, fprime)
%timeit njit_newton.py_func(func, 1.5, fprime=fprime)

33.2 µs ± 18.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.51 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


With a smart combination of closures and caching we can implement a workaround:

In [10]:
from functools import lru_cache

@lru_cache()
def newton_generator(func, fprime):
    @njit
    def njit_newton_final(x0):
        for _ in range(50):
            fder = fprime(x0)
            fval = func(x0)
            newton_step = fval / fder
            x = x0 - newton_step
            if abs(x - x0) < 1.48e-8:
                return x
            x0 = x

    return njit_newton_final

In [11]:
def newton(func, x0, fprime):
    return newton_generator(func, fprime)(x0)

%timeit newton(func, 1.5, fprime)

494 ns ± 12.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Watch out with dtypes!

https://github.com/numba/numba/issues/3993#issuecomment-485029668

In [12]:
import numpy as np

@njit
def foo():
    return np.zeros((2, 2), dtype=int)

foo()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<built-in function zeros>) with argument(s) of type(s): (tuple(int64 x 2), dtype=Function(<class 'int'>))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<built-in function zeros>)
[2] During: typing of call at <ipython-input-12-8421ff8d7be9> (5)


File "<ipython-input-12-8421ff8d7be9>", line 5:
def foo():
    return np.zeros((2, 2), dtype=int)
    ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
and
http://numba.pydata.o

In [13]:
@njit
def foo():
    return np.zeros((2, 2), dtype=np.int)

foo()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<built-in function zeros>) with argument(s) of type(s): (tuple(int64 x 2), dtype=Function(<class 'int'>))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
This error is usually caused by passing an argument of a type that is unsupported by the named function.
[1] During: resolving callee type: Function(<built-in function zeros>)
[2] During: typing of call at <ipython-input-13-8e177ef3f1ca> (3)


File "<ipython-input-13-8e177ef3f1ca>", line 3:
def foo():
    return np.zeros((2, 2), dtype=np.int)
    ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
and
http://numba.pydat

In [14]:
@njit
def foo():
    return np.zeros((2, 2), dtype=np.int_)

foo()

array([[0, 0],
       [0, 0]])

## NumPy arrays and nothing else

![Two layers](img/two-layers.png)

High level API:

* Supports complex data structures (e.g. `astropy.units` or `pint`, NumPy extensions for physical units) 
* Convert the code to normalized, simple structure that numba understands

Dangerous™ algorithms:

* Fast (easy to accelerate with `numba.njit`)
* Only cares about numbers, makes assumptions

### Example: anomaly conversion in poliastro

In [16]:
from poliastro.twobody.angles import E_to_nu
from astropy import units as u

E_to_nu(
    30 * u.deg,
    0.5 * u.one
)

<Quantity 49.79218128 deg>

In [17]:
E_to_nu(30 * u.deg, 0.5 * u.one).to(u.rad)

<Quantity 0.86903751 rad>

```python
# poliastro/core/angles.py
@jit
def E_to_nu(E, ecc):
    beta = ecc / (1 + np.sqrt((1 - ecc) * (1 + ecc)))
    nu = E + 2 * np.arctan(beta * np.sin(E) / (1 - beta * np.cos(E)))
    return nu
```

---

```python
# poliastro/twobody/angles.py
from poliastro.core.angles import E_to_nu as E_to_nu_fast

@u.quantity_input(E=u.rad, ecc=u.one)
def E_to_nu(E, ecc):
    return (E_to_nu_fast(E.to(u.rad).value, ecc.value) * u.rad).to(E.unit)
```

## Final remarks

* If anything fails, `export NUMBA_DISABLE_JIT=1`
* Try to split the function in smaller chunks until you find out what triggers object (slow) mode
* You might have to rewrite some stuff, make it less dynamic
* Keep an eye on https://numba.pydata.org/numba-doc/dev/reference/pysupported.html and https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

# Conclusions

* Numba is ✨ _awesome_ ✨ when you make it work!
* Still requires a bit of code rewrites, but the code is still _mostly pythonic_ Python
* For non-numerical code, you will probably have to find something else

# Per Python ad astra! 🚀

* Slides https://github.com/Juanlu001/talk-numba-astrodynamics
* My email <hello@juanlu.space>
* My Twitter https://twitter.com/poliastro_py

![Vega launch](img/vega.jpg)